In [35]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer

cancer_data = load_breast_cancer()
X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label,
                                                   test_size = 0.2, random_state = 156)

In [36]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state=156)
ada_clf = AdaBoostClassifier(n_estimators=100)
dt_clf = DecisionTreeClassifier()
lr_final = LogisticRegression(C = 10)

In [37]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [38]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)

In [39]:
print('knn accuracy:',accuracy_score(y_test, knn_pred))
print('rf accuracy:',accuracy_score(y_test, rf_pred))
print('ada accuracy:', accuracy_score(y_test, ada_pred))
print('dt accuracy:', accuracy_score(y_test, dt_pred))

knn accuracy: 0.9210526315789473
rf accuracy: 0.9473684210526315
ada accuracy: 0.9649122807017544
dt accuracy: 0.9298245614035088


In [40]:
pred = np.array([knn_pred, rf_pred, ada_pred, dt_pred])
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


In [41]:
lr_final.fit(pred, y_test) # y_test로 fitting 을 시킨 것 자체가 overfitting임 
lr_final_pred = lr_final.predict(pred)
accuracy_score(y_test, lr_final_pred)

0.9649122807017544

# LG유플러스 고객센터에 전화해보기

CV 셋 기반의 Stacking

In [52]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터 세트 만들기
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds 값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle = False)
    
    # 추후에 메타 모델이 사용할 학습 데이터 변환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출
        print('\t 폴드 세트',folder_counter,'시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr, y_tr)
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        # 입력된 원본 테스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장.
        test_pred[:, folder_counter] = model.predict(X_test_n)
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)
    
    # train_fold_pred 는 최정 메타 모델이 사용하는 학습 데이터, test_pred_mean 은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [53]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7);
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7);
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7);
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7);

KNeighborsClassifier model 시작
	 폴드 세트 0 시작
	 폴드 세트 1 시작
	 폴드 세트 2 시작
	 폴드 세트 3 시작
	 폴드 세트 4 시작
	 폴드 세트 5 시작
	 폴드 세트 6 시작
RandomForestClassifier model 시작
	 폴드 세트 0 시작
	 폴드 세트 1 시작
	 폴드 세트 2 시작
	 폴드 세트 3 시작
	 폴드 세트 4 시작
	 폴드 세트 5 시작
	 폴드 세트 6 시작
DecisionTreeClassifier model 시작
	 폴드 세트 0 시작
	 폴드 세트 1 시작
	 폴드 세트 2 시작
	 폴드 세트 3 시작
	 폴드 세트 4 시작
	 폴드 세트 5 시작
	 폴드 세트 6 시작
AdaBoostClassifier model 시작
	 폴드 세트 0 시작
	 폴드 세트 1 시작
	 폴드 세트 2 시작
	 폴드 세트 3 시작
	 폴드 세트 4 시작
	 폴드 세트 5 시작
	 폴드 세트 6 시작


In [55]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis = 1)
Stack_final_X_test = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis = 1)

print('원보 학습 피처 데이터 shape', X_train.shape, '월본 테스트 피처 Shape', X_test.shape)
print('스태킹 학습 피처 데이터 shape', Stack_final_X_train.shape,
     '스태킹 테스트 피처 데이터 shape', Stack_final_X_test.shape)


원보 학습 피처 데이터 shape (455, 30) 월본 테스트 피처 Shape (114, 30)
스태킹 학습 피처 데이터 shape (455, 4) 스태킹 테스트 피처 데이터 shape (455, 4)


In [57]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, stack_final)))

ValueError: Found input variables with inconsistent numbers of samples: [114, 455]